In [1]:
import pandas as pd
import numpy as np
import os, yaml, wandb

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from rl import make_transition, make_transition_for_AKI, imvt

from tqdm import tqdm

if os.getcwd()[-4:] == "code":
    os.chdir('../')

In [2]:
with open(os.path.join("./code/params.yaml")) as f:
        params = yaml.safe_load(f)

In [3]:
wandb.init(
    name="IMV_LSTM_AKI_mse",
    project="AIAKI",
    config={
        "learning_rate":params["learning_rate"],
        "epoch":params['epoch'],
        "batch_size":params["minibatch_size"],
        "n_units":params["n_units"],
        "update_freq":params["update_freq"]
    }
    )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chanreverse (dahs). Use `wandb login --relogin` to force relogin


In [4]:
train_loader, train_len = make_transition_for_AKI(params['train'],rolling_size=24,batch_size=64,istrain=True)
val_loader, val_len = make_transition_for_AKI(params['val'],rolling_size=24,batch_size=256,istrain=False)

100%|██████████| 11426/11426 [00:26<00:00, 437.63it/s]


Finished train data transition for AKI


100%|██████████| 762/762 [00:00<00:00, 952.22it/s]


Finished val or test data trnsition for AKI


In [5]:
device = 'cuda:0'
network = imvt(input_dim=params['state_dim'], output_dim=params['num_actions'], n_units=params['n_units'], device=device).to(device)
target_network = imvt(input_dim=params['state_dim'], output_dim=params['num_actions'], n_units=params['n_units'], device=device).to(device)

In [7]:
network.load_state_dict(torch.load('./checkpoint_AKI_mse/checkpoint_29.pt'))
target_network.load_state_dict(torch.load('./checkpoint_AKI_mse/checkpoint_29.pt'))

<All keys matched successfully>

In [8]:
epoch = params['epoch']
gamma = params['gamma']
optimizer = optim.Adam(network.parameters(), lr=params['learning_rate'], amsgrad=True)

update_freq = params['update_freq']
train_loss_history = []
val_loss_history = []


for i in range(29,epoch-29,1):
    loss_train = 0
    update_counter = 0
    for s,a,r,s2,t in tqdm(train_loader):
        s = s.to(device)
        a = a.to(device)
        r = r.to(device)
        s2 = s2.to(device)
        t = t.to(device)

        q,_,_ = network(s)
        q_pred = q.gather(1, a).squeeze()
        
        with torch.no_grad():
            q2,_,_ = target_network(s2)
            q2_net, _, _ = network(s2)

        q2_max = q2.gather(1, torch.max(q2_net,dim=1)[1].unsqueeze(1)).squeeze(1).detach()

        bellman_target = torch.clamp(r, max=1.0, min=0.0) + gamma * torch.clamp(q2_max.detach(), max=1.0, min=0.0)*(1-t)
        loss = F.mse_loss(q_pred, bellman_target)

        optimizer.zero_grad()
        loss.backward()
        loss_train += loss.item()
        optimizer.step()

        update_counter += 1
        if update_counter == update_freq:
            target_network.load_state_dict(network.state_dict())
            update_counter = 0

    with torch.no_grad():
        loss_val = 0

        for s,a,r,s2,t in val_loader:
            s = s.to(device)
            a = a.to(device)
            r = r.to(device)
            s2 = s2.to(device)
            t = t.to(device)

            q,_,_ = network(s)
            q2,_,_ = target_network(s2)
            q2 = q2.detach()
            q_pred = q.gather(1, a).squeeze()

            q2_net,_,_ = network(s2)
            q2_net = q2_net.detach()
            q2_max = q2.gather(1, torch.max(q2_net,dim=1)[1].unsqueeze(1)).squeeze()

            bellman_target = torch.clamp(r, max=1.0, min=0.0) + gamma * torch.clamp(q2_max.detach(), max=1.0, min=0.0)*(1-t)
            loss = F.mse_loss(q_pred, bellman_target)
            loss_val += loss.item()
            
    wandb.log({"Iter:": i, "train:":loss_train/train_len, "val:":loss_val/val_len})
    print("Iter:", i, "train:", loss_train/train_len, "val:",loss_val/val_len)
    torch.save(network.state_dict(), './checkpoint_AKI_mse/checkpoint_%i.pt'%i)
    train_loss_history.append(loss_train/train_len) 
    val_loss_history.append(loss_val/val_len)

14517it [13:35, 17.79it/s]                           


Iter: 29 train: 1.1160536702417026e-07 val: 4.3863731016518555e-10


14517it [13:47, 17.54it/s]                           


Iter: 30 train: 1.7737118274323147e-10 val: 4.242795958312263e-10


14517it [13:52, 17.43it/s]                           


...

Iter: 168 train: 3.453276292075074e-12 val: 1.4824659673073872e-10


14517it [16:09, 14.97it/s]                           


Iter: 169 train: 3.420369657941065e-12 val: 1.5024851165222023e-10


14517it [15:51, 15.25it/s]                           


Iter: 170 train: 2.744063774857611e-12 val: 1.4149813591223124e-10
